In [10]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import string
import csv

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leosh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
url = 'https://www.rottentomatoes.com/top/bestofrt/'
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, "html.parser")

In [12]:
container = soup.find('table', class_ = 'table')
container = container.find_all('a', class_ = 'unstyled articleLink')
print(type(container))

<class 'bs4.element.ResultSet'>


In [13]:
len(container)

100

In [14]:
movie_names = []
movies = container[0:50] #here we get the top 50 movies we want
for movie in movies:
    name = movie.text
    name_sp = name.strip()
    movie_names.append(name_sp)
print(movie_names)

['Black Panther (2018)', 'Avengers: Endgame (2019)', 'Us (2019)', 'Toy Story 4 (2019)', 'Lady Bird (2017)', 'Citizen Kane (1941)', 'Mission: Impossible - Fallout (2018)', 'The Wizard of Oz (1939)', 'BlacKkKlansman (2018)', 'Get Out (2017)', 'The Irishman (2019)', 'Mad Max: Fury Road (2015)', 'Spider-Man: Into the Spider-Verse (2018)', 'Casablanca (1942)', 'Moonlight (2016)', 'A Star Is Born (2018)', 'Wonder Woman (2017)', 'Dunkirk (2017)', 'Inside Out (2015)', 'The Farewell (2019)', 'A Quiet Place (2018)', 'Modern Times (1936)', 'Eighth Grade (2018)', 'It Happened One Night (1934)', 'Booksmart (2019)', 'A Night at the Opera (1935)', 'Roma (2018)', 'The Third Man (1949)', 'Coco (2017)', 'Thor: Ragnarok (2017)', 'The Shape of Water (2017)', 'Selma (2014)', 'Spotlight (2015)', 'The Godfather (1972)', 'La Grande illusion (Grand Illusion) (1938)', 'Arrival (2016)', "Singin' in the Rain (1952)", 'Snow White and the Seven Dwarfs (1937)', 'Logan (2017)', 'The Cabinet of Dr. Caligari (Das Cabin

In [15]:
movie_links = []
movies = container[:50]
for movie in movies:
    #tag = movie
    link = movie.get('href', None)
    movie_links.append(link)    
print(movie_links)

['/m/black_panther_2018', '/m/avengers_endgame', '/m/us_2019', '/m/toy_story_4', '/m/lady_bird', '/m/citizen_kane', '/m/mission_impossible_fallout', '/m/the_wizard_of_oz_1939', '/m/blackkklansman', '/m/get_out', '/m/the_irishman', '/m/mad_max_fury_road', '/m/spider_man_into_the_spider_verse', '/m/1003707-casablanca', '/m/moonlight_2016', '/m/a_star_is_born_2018', '/m/wonder_woman_2017', '/m/dunkirk_2017', '/m/inside_out_2015', '/m/the_farewell_2019', '/m/a_quiet_place_2018', '/m/modern_times', '/m/eighth_grade', '/m/it_happened_one_night', '/m/booksmart', '/m/1015002-night_at_the_opera', '/m/roma_2018', '/m/the_third_man', '/m/coco_2017', '/m/thor_ragnarok_2017', '/m/the_shape_of_water_2017', '/m/selma', '/m/spotlight_2015', '/m/godfather', '/m/la_grande_illusion', '/m/arrival_2016', '/m/singin_in_the_rain', '/m/1048445-snow_white_and_the_seven_dwarfs', '/m/logan_2017', '/m/the_cabinet_of_dr_caligari', '/m/double_indemnity', '/m/the_favourite_2018', '/m/the_big_sick', '/m/spider_man_fa

In [16]:
for i in range(len(movie_links)):
    movie_links[i] = 'https://www.rottentomatoes.com'+ movie_links[i]
print(movie_links)

['https://www.rottentomatoes.com/m/black_panther_2018', 'https://www.rottentomatoes.com/m/avengers_endgame', 'https://www.rottentomatoes.com/m/us_2019', 'https://www.rottentomatoes.com/m/toy_story_4', 'https://www.rottentomatoes.com/m/lady_bird', 'https://www.rottentomatoes.com/m/citizen_kane', 'https://www.rottentomatoes.com/m/mission_impossible_fallout', 'https://www.rottentomatoes.com/m/the_wizard_of_oz_1939', 'https://www.rottentomatoes.com/m/blackkklansman', 'https://www.rottentomatoes.com/m/get_out', 'https://www.rottentomatoes.com/m/the_irishman', 'https://www.rottentomatoes.com/m/mad_max_fury_road', 'https://www.rottentomatoes.com/m/spider_man_into_the_spider_verse', 'https://www.rottentomatoes.com/m/1003707-casablanca', 'https://www.rottentomatoes.com/m/moonlight_2016', 'https://www.rottentomatoes.com/m/a_star_is_born_2018', 'https://www.rottentomatoes.com/m/wonder_woman_2017', 'https://www.rottentomatoes.com/m/dunkirk_2017', 'https://www.rottentomatoes.com/m/inside_out_2015',

In [17]:
genre_lst_rot = []
for i in movie_links:
    #genre
    m_page_rot = requests.get(i, headers = headers)
    m_soup_rot = BeautifulSoup(m_page_rot.content, 'lxml')
    genre_rot = m_soup_rot.find('div', class_ = 'meta-value genre').text.lower().strip()
    genres_rot = re.sub('\s+','',genre_rot).split(',')
    genre_lst_rot.append(genres_rot)
print(genre_lst_rot)

[['fantasy', 'action', 'adventure'], ['fantasy', 'adventure', 'scifi', 'action'], ['horror', 'mysteryandthriller'], ['fantasy', 'kidsandfamily', 'animation', 'adventure', 'comedy'], ['drama', 'comedy'], ['drama'], ['adventure', 'mysteryandthriller', 'action'], ['musical', 'kidsandfamily', 'fantasy'], ['drama', 'comedy', 'crime'], ['horror', 'mysteryandthriller', 'comedy'], ['drama', 'crime'], ['adventure', 'action'], ['fantasy', 'comedy', 'kidsandfamily', 'animation', 'adventure', 'action'], ['drama'], ['drama'], ['drama', 'music', 'romance'], ['adventure', 'action', 'fantasy'], ['drama', 'war', 'history'], ['comedy', 'fantasy', 'kidsandfamily', 'animation'], ['drama', 'comedy'], ['horror', 'mysteryandthriller'], ['comedy'], ['drama', 'comedy'], ['romance'], ['comedy'], ['comedy'], ['drama'], ['mysteryandthriller'], ['music', 'kidsandfamily', 'animation', 'adventure', 'comedy'], ['fantasy', 'comedy', 'scifi', 'adventure', 'action'], ['fantasy', 'romance'], ['drama', 'history'], ['drama

In [18]:
#you will notice that we got critic reviews instead of user reviews
#but going back, you will see critic reviews and user reviews uses exactlt the same tag, we cannot tell them apart
#so we go back and use the find_all function, this will get us 2 results for each movie, 1 for critic review, 1 for user review
#we only want the second one
review_links = []
genre_lst_rot = []
for i in movie_links:
    #page = requests.get(i, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    review_link = i + '/reviews?type=user'
    review_links.append(review_link)
    m_page_rot = requests.get(i, headers = headers)
    m_soup_rot = BeautifulSoup(m_page_rot.content, 'lxml')
    genre_rot = m_soup_rot.find('div', class_ = 'meta-value genre').text.lower().strip()
    genres_rot = re.sub('\s+','',genre_rot).split(',')
    genre_lst_rot.append(genres_rot)
print(review_links) 
print(genre_lst_rot)

['https://www.rottentomatoes.com/m/black_panther_2018/reviews?type=user', 'https://www.rottentomatoes.com/m/avengers_endgame/reviews?type=user', 'https://www.rottentomatoes.com/m/us_2019/reviews?type=user', 'https://www.rottentomatoes.com/m/toy_story_4/reviews?type=user', 'https://www.rottentomatoes.com/m/lady_bird/reviews?type=user', 'https://www.rottentomatoes.com/m/citizen_kane/reviews?type=user', 'https://www.rottentomatoes.com/m/mission_impossible_fallout/reviews?type=user', 'https://www.rottentomatoes.com/m/the_wizard_of_oz_1939/reviews?type=user', 'https://www.rottentomatoes.com/m/blackkklansman/reviews?type=user', 'https://www.rottentomatoes.com/m/get_out/reviews?type=user', 'https://www.rottentomatoes.com/m/the_irishman/reviews?type=user', 'https://www.rottentomatoes.com/m/mad_max_fury_road/reviews?type=user', 'https://www.rottentomatoes.com/m/spider_man_into_the_spider_verse/reviews?type=user', 'https://www.rottentomatoes.com/m/1003707-casablanca/reviews?type=user', 'https://

In [20]:
n = 0
tag_lst = []
wordlist = []
for i in range(len(review_links)):
    review_page = requests.get(review_links[i], headers = headers)
    soup = BeautifulSoup(review_page.content, "html.parser")
    tag_p = soup.find_all('div', class_ = 'audience-reviews__review-wrap') #find_all to return a list
    tag_f = soup.find('p', class_ = 'audience-reviews__review js-review-text clamp clamp-8 js-clamp')
    tag_r = re.sub("<.?>", "", tag_f.text)
    tag_r = (tag_r.strip()).lower()
    tag = re.sub(r'[^\w\s]', '', tag_r)
    text_token = word_tokenize(tag)
    review3 = [word for word in text_token if not word in stopwords.words()]    
    n = n + 1
    print('Movie', n)
    print('Total user reviews:', len(tag_p))
    top_50 = tag[:50]
    print(len(tag_p))#some of the movies don't have 50 reviews, but that is fine  
    print(tag)
    tag_lst.append(tag)
    wordlist += review3

print(wordlist)
print(review3)



Movie 1
Total user reviews: 10
10
it was absolutely appaling i have never been more bored or ashamed of marvel in my entire life dreadful movie would never watch again or recommend
Movie 2
Total user reviews: 10
10
i dont know how everyone at marvel pulled this off but im so glad that they did as a huge marvel fan this movie was perfect and i couldnt be happier the cast was great  the russos have directed my favourite marvel movies  the action was great as well captain america is my favourite superhero ever  and to see how perfectly they wrapped his story was very gratifying  this movie was everything i had hoped for and more well done to everyone at marvel for the amazing work over the last 12 years  im looking forward to where the mcu goes from here
Movie 3
Total user reviews: 10
10
i was so disapointed great performances by the actors and yes i do understand what peele was trying to convey it just left me feeling so utterly disapointed
Movie 4
Total user reviews: 10
10
this score is

Movie 23
Total user reviews: 10
10
a unique coming of age film set in the modern world awkward but touching
Movie 24
Total user reviews: 10
10
afi 100 greatest films  35 this is a film i knew nothing about and went into this with only the very short description on the tcm listing as background knowledge the brief synopsis under the title listing made this film sound more like a mysterysuspense when i read it so i was surprised and thrown off by the quirky and playful tone early into the picture not realizing i was actually watching one of the original romcoms it isnt hard to see why this was the first film to take home the big four at the academy awards as the directing and acting from the two leads was so in sync and i imagine quite groundbreaking for that time as elements of this film are still seen prevalently in romantic comedies today definitely not my favorite film or genre but one definitely that would be hard not to appreciate and respect
Movie 25
Total user reviews: 10
10
a gr

Movie 43
Total user reviews: 10
10
it took me a little while to get into it but it eventually pulled me in and i laughed several times i went into it without knowing much beforehand and i had no idea it was based on the reallife story of the lead actor until the pictures at the end i thought ray romano was particularly great in this it has some charm to it
Movie 44
Total user reviews: 10
10
very fun to watch i like how this is similiar to the spiderman from the comic a fresh movie to watch
Movie 45
Total user reviews: 10
10
a classic in its own right et has everything a family movie needs and more
Movie 46
Total user reviews: 10
10
it was flashy but i liked less the more i thought about it
Movie 47
Total user reviews: 10
10
the dialog in these older films is always pretty good eve was really sketchy from the get go here but they didnt give it enough legs as to what she may be up to the films didnt really pick up till the final act and it was all like blam blackmail deceit back stabbing

In [11]:
#Rotten_list[title] = []
#Rotten_list[title] = {'link': review_links, 'genre': genre_lst_rot, 'reviews': tag_lst}
#print(Rotten_list[title])
print(review_links)
print(genre_lst_rot)
print(tag_lst)

['https://www.rottentomatoes.com/m/black_panther_2018/reviews?type=user', 'https://www.rottentomatoes.com/m/avengers_endgame/reviews?type=user', 'https://www.rottentomatoes.com/m/us_2019/reviews?type=user', 'https://www.rottentomatoes.com/m/toy_story_4/reviews?type=user', 'https://www.rottentomatoes.com/m/lady_bird/reviews?type=user', 'https://www.rottentomatoes.com/m/citizen_kane/reviews?type=user', 'https://www.rottentomatoes.com/m/mission_impossible_fallout/reviews?type=user', 'https://www.rottentomatoes.com/m/the_wizard_of_oz_1939/reviews?type=user', 'https://www.rottentomatoes.com/m/blackkklansman/reviews?type=user', 'https://www.rottentomatoes.com/m/get_out/reviews?type=user', 'https://www.rottentomatoes.com/m/the_irishman/reviews?type=user', 'https://www.rottentomatoes.com/m/mad_max_fury_road/reviews?type=user', 'https://www.rottentomatoes.com/m/spider_man_into_the_spider_verse/reviews?type=user', 'https://www.rottentomatoes.com/m/1003707-casablanca/reviews?type=user', 'https://

In [12]:
fields = ['Link', 'Genre', 'Reviews']  
rows = [review_links, genre_lst_rot, tag_lst]
filename = "records.csv"
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(fields)  
        
    # writing the data rows  
    csvwriter.writerows(rows) 

In [21]:
print(wordlist)

['absolutely', 'appaling', 'never', 'bored', 'ashamed', 'marvel', 'entire', 'life', 'dreadful', 'movie', 'would', 'never', 'watch', 'recommend', 'dont', 'know', 'everyone', 'marvel', 'pulled', 'glad', 'huge', 'marvel', 'fan', 'movie', 'perfect', 'couldnt', 'happier', 'cast', 'great', 'russos', 'directed', 'favourite', 'marvel', 'movies', 'action', 'great', 'well', 'captain', 'america', 'favourite', 'superhero', 'ever', 'see', 'perfectly', 'wrapped', 'story', 'gratifying', 'movie', 'everything', 'hoped', 'well', 'done', 'everyone', 'marvel', 'amazing', 'work', 'last', '12', 'years', 'looking', 'forward', 'mcu', 'goes', 'disapointed', 'great', 'performances', 'actors', 'yes', 'understand', 'peele', 'trying', 'convey', 'left', 'feeling', 'utterly', 'disapointed', 'score', 'definitely', 'high', 'extremely', 'disappointed', 'toy', 'story', '4', 'point', 'ended', 'perfectly', '3', 'happy', 'together', '4', 'erases', 'everything', 'movies', 'like', 'screw', 'friends', 'screw', 'child', 'date'

In [22]:
word_count = 0
for word in wordlist:
        words = word.strip().split()            # for each line split words
        word_count = word_count + len(words)
print(word_count)

1728


In [23]:
set_wordlist = set(wordlist)

In [29]:
#rows = [review_links, genre_lst_rot, tag_lst]
import pandas as pd

pd.DataFrame({'Links': review_links, 'Genre': genre_lst_rot, 'Reviews':tag_lst}).to_csv('abcd.csv', index=False)
(pd.DataFrame({'Links': review_links, 'Genre': genre_lst_rot, 'Reviews':tag_lst}, columns=['Links','Genre','Reviews'])
   .to_csv('abcd.csv', index=False))

In [41]:
import sqlite3
conn = sqlite3.connect('MovieReviewDatabase.sqlite')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS MovieReviewTable_Rot')
cur.execute('CREATE TABLE MovieReviewTable_Rot (Links VARCHAR(255), Genre VARCHAR(255), Reviews VARCHAR(255))')
moviefilehandle = open('abcd.csv','r')
insertcmd = 'INSERT INTO MovieReviewTable_Rot (Links, Genre, Reviews) VALUES (?,?,?)'

for movie in moviefilehandle:
    movie = movie.rstrip()
    moviedata = movie.split(',')
    cur.execute(insertcmd, (moviedata[0],moviedata[1],moviedata[2]))
    conn.commit()
cur.close()
moviefilehandle.close()

cur = conn.cursor()
print("MovieReviewTable_Rot: ")
cur.execute('SELECT Links, Genre, Reviews FROM MovieReviewTable_Rot')
for row in cur:
    print(row)

cur.close()


IndexError: list index out of range